In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

REPO_NAME = 'draft-assistant'
CWD = str(os.getcwd())
REPO_DIR = CWD[:CWD.find(REPO_NAME)+len(REPO_NAME)]
sys.path.insert(0,REPO_DIR)

In [7]:
import nfl_data_py as nflreadr

In [9]:
players = nflreadr.import_players()

In [21]:
pbp_data = nflreadr.import_pbp_data(years = range(2016,2023))

2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


In [176]:
from src.league.league import League

l = League(
    draft_position=12,
    n_teams=12,
    end_week=17
)

In [189]:
offensive_stats = l.offense_fantasy_stats(pbp_data)
offensive_stats = offensive_stats.merge(players[['gsis_id','display_name','position']],
                      left_on = ['player_id'],
                      right_on = ['gsis_id'],
                      how = 'inner')
POS = ['QB','WR','RB','TE']
offensive_stats = offensive_stats[offensive_stats['position'].isin(POS)]
offensive_stats = offensive_stats.drop(['gsis_id'],axis = 'columns')

c:\Users\zacha\Documents\GitHub\draft-assistant\src\stats\offense.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pbp_data['receiver'] = 1
c:\Users\zacha\Documents\GitHub\draft-assistant\src\stats\offense.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pbp_data['desc'] = 1


In [190]:
gb_stats = offensive_stats.drop(['posteam','week'], axis = 'columns').groupby(
    ['season','player_id','display_name','position']
)
season_stats = gb_stats.agg({
    'FP':['sum','mean','count','median']
}).reset_index()
cols = [''.join(list(x)) for x in season_stats.columns]
season_stats.columns = cols

In [214]:
# Calculate retrospective VOR
from scipy.stats import zscore

season = 2022
metric = 'mean'
filter_metric = 'sum'
starters = 12
position = 'QB'

mask = (season_stats['position'] == position) & (season_stats['season'] == season)
view = season_stats[mask]
view['rank'] = view[f'FP{filter_metric}'].rank(method = 'min', ascending=False)
view = view.sort_values('rank')
view = view[view['rank'] <= 2 * starters]
view['z'] = view[[f'FP{metric}']].apply(zscore).iloc[:,0]

view

C:\Users\zacha\AppData\Local\Temp\ipykernel_32612\546708962.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  view['rank'] = view[f'FP{filter_metric}'].rank(method = 'min', ascending=False)


,season,player_id,display_name,position,FPsum,FPmean,FPcount,FPmedian,rank,z
3420,2022,00-0033873,Patrick Mahomes,QB,470.40,29.400000,16,29.800,1.0,2.008452
3498,2022,00-0034857,Josh Allen,QB,446.05,29.736667,15,30.850,2.0,2.094304
3640,2022,00-0036389,Jalen Hurts,QB,414.30,29.592857,14,30.525,3.0,2.057632
3652,2022,00-0036442,Joe Burrow,QB,403.70,26.913333,15,25.750,4.0,1.374339
3300,2022,00-0030565,Geno Smith,QB,356.95,22.309375,16,22.575,5.0,0.200305
3713,2022,00-0036971,Trevor Lawrence,QB,350.45,21.903125,16,21.425,6.0,0.096710
3286,2022,00-0029604,Kirk Cousins,QB,349.80,21.862500,16,21.450,7.0,0.086350
3707,2022,00-0036945,Justin Fields,QB,342.40,22.826667,15,22.300,8.0,0.332218
3376,2022,00-0033106,Jared Goff,QB,334.50,20.906250,16,19.825,9.0,-0.157499
3573,2022,00-0035710,Daniel Jones,QB,333.05,20.815625,16,20.075,10.0,-0.180609


In [183]:
view = season_stats[season_stats['position']=='QB']
view.sort_values('FPsum',ascending=False)

,season,player_id,display_name,position,FPsum,FPmean,FPcount,FPmedian
1591,2018,00-0033873,Patrick Mahomes,QB,494.05,30.878125,16,29.925
2722,2020,00-0034857,Josh Allen,QB,473.00,29.562500,16,31.000
3694,2022,00-0033873,Patrick Mahomes,QB,470.40,29.400000,16,29.800
2195,2019,00-0034796,Lamar Jackson,QB,462.95,30.863333,15,33.000
2757,2020,00-0035228,Kyler Murray,QB,450.45,28.153125,16,30.500
...,...,...,...,...,...,...,...,...
622,2017,00-0026197,Chad Henne,QB,-0.50,-0.250000,2,-0.250
1521,2018,00-0033119,Jacoby Brissett,QB,-0.60,-0.200000,3,-0.400
3221,2021,00-0034438,Logan Woodside,QB,-0.60,-0.150000,4,-0.100
1890,2019,00-0031407,Blake Bortles,QB,-0.75,-0.375000,2,-0.375
